In [ ]:
## Before we start tunning let's see how the models perform with few shots


# Models
# We use instruct models only as i have no idea how I would "prompt" completion models
# with current setting
models = [
    "NousResearch/Llama-2-7b-chat-hf",
    "mistralai/Mistral-7B-Instruct-v0.2",
    "codellama/CodeLlama-7b-Instruct-hf",
]




In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("codellama/CodeLlama-7b-Instruct-hf")
from transformers import AutoModelForCausalLM
x = tokenizer.apply_chat_template(
    [{"role": "system", "content": "Generate a fitting name for the provided function"},{"role": "user", "content": "Create a code which sums up two numbers"}],
    tokenize=True, return_tensors="pt"
)


/workspace/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

No chat template is defined for this tokenizer - using the default template for the CodeLlamaTokenizerFast class. If the default is not appropriate for your model, please set `tokenizer.chat_template` to an appropriate template. See https://huggingface.co/docs/transformers/main/chat_templating for more information.



In [3]:
import torch
model  = AutoModelForCausalLM.from_pretrained("codellama/CodeLlama-7b-Instruct-hf", torch_dtype=torch.bfloat16).to("cuda")

Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.18s/it]


In [5]:
result = model.generate(x.to("cuda"), max_length=100, num_return_sequences=1)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [8]:
from datasets import load_from_disk
dst = load_from_disk("../s3/hf/datasets/python_code_cleaned")

In [15]:
# Take first 100 examples
examples = dst["train"].select(range(100))

In [20]:
examples

Dataset({
    features: ['source_code', 'function_name'],
    num_rows: 100
})

In [52]:
# Tokenize
def apply_template(x):
    return {"text": tokenizer.apply_chat_template(
        [{"role": "system", "content": "Generate a fitting name for the provided function"},{"role": "user", "content": x["source_code"]}],
        tokenize=False)
    }
examples_tokenized = examples.map(apply_template)

In [65]:
print(examples_tokenized[0]["text"])

<s>[INST] <<SYS>>
Generate a fitting name for the provided function
<</SYS>>

def x(self):
    import scipy
    start = self._start_time
    seg = 1 - self.segmentation.astype(np.int8)
    self.stats['low level object voxels'] = np.sum(seg)
    self.stats['low level image voxels'] = np.prod(seg.shape)
    seg_border = scipy.ndimage.filters.laplace(seg, mode='constant')
    logger.debug('seg_border: %s', scipy.stats.describe(seg_border, axis=None))
    seg_border[seg_border != 0] = 1
    logger.debug('seg_border: %s', scipy.stats.describe(seg_border, axis=None))
    boundary_dilatation_distance = self.segparams[
        'boundary_dilatation_distance']
    seg = scipy.ndimage.morphology.binary_dilation(seg_border, np.ones([
        boundary_dilatation_distance * 2 + 1, boundary_dilatation_distance *
        2 + 1, boundary_dilatation_distance * 2 + 1]))
    if self.keep_temp_properties:
        self.temp_msgc_lowres_discontinuity = seg
    else:
        self.temp_msgc_lowres_discontinuit

In [69]:
tokenizer.decode(model.generate(tokenizer(examples_tokenized[0]["text"], return_tensors="pt")["input_ids"].to("cuda"), num_return_sequences=1, max_new_tokens=200)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'<s><s> [INST] <<SYS>>\nGenerate a fitting name for the provided function\n<</SYS>>\n\ndef x(self):\n    import scipy\n    start = self._start_time\n    seg = 1 - self.segmentation.astype(np.int8)\n    self.stats[\'low level object voxels\'] = np.sum(seg)\n    self.stats[\'low level image voxels\'] = np.prod(seg.shape)\n    seg_border = scipy.ndimage.filters.laplace(seg, mode=\'constant\')\n    logger.debug(\'seg_border: %s\', scipy.stats.describe(seg_border, axis=None))\n    seg_border[seg_border != 0] = 1\n    logger.debug(\'seg_border: %s\', scipy.stats.describe(seg_border, axis=None))\n    boundary_dilatation_distance = self.segparams[\n        \'boundary_dilatation_distance\']\n    seg = scipy.ndimage.morphology.binary_dilation(seg_border, np.ones([\n        boundary_dilatation_distance * 2 + 1, boundary_dilatation_distance *\n        2 + 1, boundary_dilatation_distance * 2 + 1]))\n    if self.keep_temp_properties:\n        self.temp_msgc_lowres_discontinuity = seg\n    else:\n   